In [28]:
!pip install django pyngrok


In [29]:
import os

os.system("django-admin startproject event_project")
os.chdir("event_project")
os.system("python manage.py startapp events")


0

In [30]:
settings_path = "event_project/settings.py"

with open(settings_path, "r") as file:
    content = file.read()

content = content.replace(
    "INSTALLED_APPS = [",
    "INSTALLED_APPS = [\n    'events',"
).replace(
    "ALLOWED_HOSTS = []",
    "ALLOWED_HOSTS = ['*']"
)

with open(settings_path, "w") as file:
    file.write(content)


In [31]:
model_code = '''
from django.db import models
from django.contrib.auth.models import User

class Event(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    date = models.DateTimeField()
    location = models.CharField(max_length=255)

    def __str__(self):
        return self.title

class Registration(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)
    event = models.ForeignKey(Event, on_delete=models.CASCADE)
    registered_at = models.DateTimeField(auto_now_add=True)

    class Meta:
        unique_together = ('user', 'event')
'''
with open("events/models.py", "w") as f:
    f.write(model_code)


In [32]:
!python manage.py makemigrations
!python manage.py migrate


Migrations for 'events':
  events/migrations/0001_initial.py
    + Create model Event
    + Create model Registration
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, events, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_gr

In [33]:
import django
import threading
import os

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'event_project.settings')
django.setup()

def create_superuser():
    from django.contrib.auth.models import User
    if not User.objects.filter(username='admin').exists():
        User.objects.create_superuser('admin', 'admin@example.com', 'admin123')
        print("✅ Superuser created.")
    else:
        print("⚠️ Superuser already exists.")

thread = threading.Thread(target=create_superuser)
thread.start()
thread.join()


⚠️ Superuser already exists.


In [34]:
# views.py
with open("events/views.py", "w") as f:
    f.write('''
from django.http import JsonResponse, HttpResponse
from .models import Event

def home(request):
    return HttpResponse("<h1>🎉 Welcome to the Event API</h1><p>Visit <a href='/events/'>/events/</a></p>")

def event_list(request):
    events = list(Event.objects.values())
    return JsonResponse(events, safe=False)
''')

# urls.py for app
with open("events/urls.py", "w") as f:
    f.write('''
from django.urls import path
from .views import event_list, home

urlpatterns = [
    path('', home),
    path('events/', event_list),
]
''')

# urls.py for project
with open("event_project/urls.py", "w") as f:
    f.write('''
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('events.urls')),
]
''')


In [35]:
# Kill previous Django port
!kill -9 $(lsof -t -i:8000) 2>/dev/null

from pyngrok import ngrok
import time
import threading

# Set your ngrok token
ngrok.set_auth_token("2zJ138qzNiWEiEB3d4BIPgA8YC1_83CiiAVsnpJ2LjF7EQteg")  # Replace with your token

# Start Django server
def run_server():
    os.system("python manage.py runserver 0.0.0.0:8000")

thread = threading.Thread(target=run_server)
thread.start()

time.sleep(3)  # Wait for server to start

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("🚀 Your app is live at:", public_url)


🚀 Your app is live at: NgrokTunnel: "https://04c8-34-80-183-243.ngrok-free.app" -> "http://localhost:8000"
